**This is a simple Deobfuscation scipt that deobfuscates the minecraft assets/objects folder to make sounds and hiden textures available**

In [1]:
from pathlib import Path
import re
import json
import glob
from collections.abc import Callable
from os.path import expanduser

class Deobfuscator:
    def __init__(self, mcDir: Path, version: str) -> None:
        if not mcDir.exists() or not mcDir.is_dir():
            raise FileNotFoundError(f"Directory {mcDir} does not exist")
        if re.match(r"\d+\.\d+\.\d+", version) is None:
            raise ValueError(f"Invalid version: {version}")
        
        self.mcDir = mcDir
        self.version = version
        
    def load(self) -> None:
        index_location = (self.mcDir / "assets" / "indexes" / f"{self.get_major_version(self.version)}.json")
        print(index_location)
        index_text = index_location.read_text()
        self.json = json.loads(index_text)
        
        self.files = {}
        for file in glob.glob(str(self.mcDir / "assets" / "objects" / "**"), recursive=True):
            path = Path(file)
            if path.is_file():
                self.files[path.name] = path
        
    def deobfuscate(self, filter: Callable[[Path], bool]) -> str:
        self.mapped_files = {}
        objects = self.json["objects"]
        for key in objects:
            if filter(Path(key)):
                object = objects[key]
                hash = object["hash"]
                self.mapped_files[hash] = key
                
    def write(self, output: Path) -> None:
        output.mkdir(parents=True, exist_ok=True)
        
        for hash in self.mapped_files:
            path = output / self.mapped_files[hash]
            path.parent.mkdir(parents=True, exist_ok=True)
            
            with open(path, "wb") as f:
                f.write((self.mcDir / "assets" / "objects" / self.files[hash]).read_bytes())
                
        
        
        
    @staticmethod
    def get_major_version(version: str) -> str:
        return re.match(r"(\d+\.\d+)", version).group(1)

home = Path(expanduser("~"))

mcdir_raw = input("Enter the path to the Minecraft directory:")
if len(mcdir_raw) == 0:
    mcdir = home / "AppData" / "Roaming" / ".minecraft"
else:
    mcdir = Path(mcdir_raw)
    
mcver = input("Enter the Minecraft version(s):")
if len(mcver) == 0:
    mcver = "1.16.5,1.18.2,1.19.4"
    
extension_filter = input("Enter file filter (regex) (leave blank for all):")
output_dir_raw = input("Enter the output directory: ")
if len(output_dir_raw) == 0:
    output_dir = Path("output")
else:
    output_dir = Path(output_dir_raw)

version_list = mcver.split(",")

for version in version_list:
    do = Deobfuscator(mcdir, version)
    do.load()
    do.deobfuscate(lambda f: re.match(extension_filter, f.name) or not extension_filter)
    do.write(output_dir / (version + "_object"))


C:\Users\Rick\AppData\Roaming\.minecraft\assets\indexes\1.16.json
